In [74]:
pip install konlpy

In [75]:
import pandas as pd

# 데이터는 이미 Okt
data_path = './drive/MyDrive/Colab Notebooks/jejuRecommand/data/after_preprocessing.csv'

df = pd.read_csv(data_path)

In [76]:
df['platform'].unique()

array(['naver', 'brunch'], dtype=object)

In [77]:
# 네이버와 브런치로 데이터 프레임 나누기
df_naver = df[df['platform']=='naver']
print(len(df_naver))
df_naver.head()

350001


,source,content,postdate,platform
0,휴애리 자연생활공원,언니 평소 가보고 싶어 하던 휴애리 축제 다녀왔어요 지금 한참 만발 해서 꽃길 걷다...,20211217,naver
1,휴애리 자연생활공원,안녕하세요 아델라 입니다 이번 포스팅 휴애리 자연생활공원 사진 많은 이었어요 저희 ...,20220227,naver
2,휴애리 자연생활공원,제주도 휴애리 자연생활공원 제주 매화축제 구경 제주 서귀포시 남원읍 매일 연중 입장...,20220306,naver
3,휴애리 자연생활공원,안녕하세요 량애 절반 지나가고 있네요 아직도 머물러 있어요 제주도 여행 다녀온지 얼...,20220127,naver
4,휴애리 자연생활공원,서귀포 가볼만 휴애리 자연생활공원 동백 포토 진짜 제주도 가서 이렇게 동백 미친듯이...,20220104,naver


In [78]:
df_brunch = df[df['platform'] == 'brunch'].reset_index()
del df_brunch['index']
print(len(df_brunch))
df_brunch.head()

11705


,source,content,postdate,platform
0,휴애리 자연생활공원,가지 눈송이 다시 쌓였 햇살 하나 얼굴 밀면 잎사귀 밀어내니 하얀 겉옷 화려히 차려...,20170220,brunch
1,휴애리 자연생활공원,먼저 피는 매화나무 가장 먼저 알리는 꽃나무 매화나무 한라산 배경 아름다운 풍경 보...,20220209,brunch
2,휴애리 자연생활공원,사계절 내내 다른 풍경 선사 하는 제주 언제 방문 해도 즐거운 여행지 입니다 요즘 ...,20220103,brunch
3,휴애리 자연생활공원,모처럼 여행 춥다 웅크리고 있을 없다 추운 겨울 신나게 물놀이 즐기고 싶은 위해 준...,20210210,brunch
4,휴애리 자연생활공원,제주 핑크 리가 예쁘게 물결 치는 계절 어디 가야 로맨틱 핑크 뮬리 물결 만날 있는...,20211001,brunch


In [7]:
# 네이버 vocab 만들기
naver_vocab = {}
sentences = df_naver['content']
for sentence in sentences:
  li = sentence.split()
  for i in li:
    if i not in naver_vocab:
      naver_vocab[i] = 0
    naver_vocab[i] += 1
len(naver_vocab)

1345824

In [8]:
# 빈도수가 높은 순서로 정렬

naver_vocab_sorted = sorted(naver_vocab.items(), key = lambda x: -x[1])

print(naver_vocab_sorted[:50])

# 너무 쓸데 없는 동사가 많이 나온다.

[('제주', 1675616), ('있는', 1131084), ('하는', 870529), ('제주도', 833076), ('코스', 657798), ('사진', 648349), ('사람', 599738), ('있다', 568649), ('시간', 558359), ('입니다', 547328), ('너무', 537924), ('바다', 490506), ('오름', 485342), ('여행', 454034), ('했다', 452153), ('제주특별자치도', 439313), ('생각', 428073), ('카페', 427380), ('우리', 415513), ('제주시', 399709), ('한다', 390639), ('많이', 381248), ('되어', 354986), ('함께', 343803), ('마을', 340049), ('하고', 328452), ('합니다', 328344), ('해서', 326989), ('바로', 325304), ('정말', 317820), ('아이', 316351), ('정도', 308930), ('좋은', 308072), ('하나', 298959), ('서귀포시', 297312), ('하여', 295612), ('때문', 290547), ('모습', 289123), ('여기', 288004), ('체험', 282424), ('이렇게', 275675), ('있습니다', 274161), ('시작', 269606), ('다시', 266979), ('호텔', 258550), ('풍경', 256980), ('있어서', 244905), ('같은', 239887), ('오늘', 238698), ('따라', 238409)]


In [9]:
# 명사 외 제거
from konlpy.tag import Okt

okt = Okt()
naver_vocab_without_verb = []
for i in naver_vocab_sorted:
  voc = okt.nouns(i[0])
  if voc:
    naver_vocab_without_verb.append(voc[0])

In [10]:
print(naver_vocab_without_verb[:50])

['제주', '제주도', '코스', '사진', '사람', '시간', '바다', '오름', '여행', '제주', '생각', '카페', '우리', '제주시', '마을', '바로', '정말', '아이', '정도', '하나', '서귀포시', '때문', '모습', '여기', '체험', '시작', '다시', '호텔', '풍경', '오늘', '위치', '다른', '올레', '가장', '위해', '보고', '도착', '입구', '맛집', '바람', '한라산', '거리', '이용', '서귀포', '추천', '다음', '날씨', '진짜', '조금', '방문']


In [31]:
# 브런치 vocab 만들기
brunch_vocab = {}
sentences = df_brunch['content']

for sentence in sentences:
  if type(sentence)==float:   # nan 값 제거
    continue
  li = sentence.split()
  for i in li:
    if i not in brunch_vocab:
      brunch_vocab[i] = 0
    brunch_vocab[i] += 1
len(brunch_vocab)

135022

In [32]:
# 빈도수가 높은 순서로 정렬

brunch_vocab_sorted = sorted(brunch_vocab.items(), key = lambda x: -x[1])

print(brunch_vocab_sorted[:50])

[('제주', 89909), ('있는', 52289), ('하는', 48652), ('있다', 45718), ('했다', 33619), ('사람', 32514), ('제주도', 32075), ('코스', 31954), ('한다', 30630), ('여행', 30474), ('바다', 27977), ('생각', 24828), ('시간', 23797), ('오름', 21814), ('함께', 18380), ('우리', 16551), ('시작', 16451), ('사진', 15278), ('되어', 15105), ('마을', 14799), ('때문', 14586), ('올레', 14102), ('하나', 13957), ('모습', 13942), ('아이', 13753), ('있었다', 13717), ('입니다', 13575), ('정도', 13476), ('가장', 12736), ('다시', 12669), ('하고', 12354), ('풍경', 12186), ('같은', 12006), ('다른', 11974), ('이었다', 11676), ('올레길', 11663), ('바람', 11590), ('마음', 11332), ('한라산', 11063), ('많은', 11054), ('카페', 11032), ('따라', 10965), ('있어', 10908), ('자연', 10881), ('많이', 10829), ('없는', 10632), ('좋은', 10493), ('위해', 10465), ('해변', 10416), ('바로', 10158)]


In [33]:
# 동사 제거
from konlpy.tag import Okt

okt = Okt()
brunch_vocab_without_verb = []
for i in brunch_vocab_sorted:
  voc = okt.nouns(i[0])
  if voc:
    brunch_vocab_without_verb.append(voc[0])

In [34]:
print(brunch_vocab_without_verb[:50])

['제주', '사람', '제주도', '코스', '여행', '바다', '생각', '시간', '오름', '우리', '시작', '사진', '마을', '때문', '올레', '하나', '모습', '아이', '정도', '가장', '다시', '풍경', '다른', '올레길', '바람', '마음', '한라산', '카페', '자연', '위해', '해변', '바로', '호텔', '지금', '위치', '공간', '나무', '거리', '이야기', '이름', '우도', '조금', '숙소', '보고', '가지', '길이', '하늘', '오늘', '서귀포', '도착']


In [64]:
# 인스타

# 데이터는 이미 Okt
data_path = './drive/MyDrive/Colab Notebooks/jejuRecommand/data/insta.csv'

df = pd.read_csv(data_path)
df.head()

,id,geo_tag,date,img_url,content,hash_tag,like,url
0,merri_sweet_home_,금능해수욕장,2022년 3월 20일,https://scontent-gmp1-1.cdninstagram.com/v/t51...,⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀결혼식끝난지 얼마안되고 바로 4주년앞으로는 매년 3월에...,[],110,https://www.instagram.com/p/CbUOVC1rimP/
1,park_bin,서귀포,2022년 3월 20일,https://scontent-gmp1-1.cdninstagram.com/v/t51...,제주도 바다는 다 예뻐 🌊........#서귀포 #제주여행 #제주도 #주말 #제주주...,"['서귀포', '제주여행', '제주도', '주말', '제주주말', '제주도여행', ...",333,https://www.instagram.com/p/CbUjxKlpWG1/
2,luv_mingg,제주 산방산,2022년 3월 20일,https://scontent-gmp1-1.cdninstagram.com/v/t51...,유채꽃은 산방산으로 보러 가세요🌼💛,[],575,https://www.instagram.com/p/CbUw9fbBuO7/
3,d_in____,사계 해변,2022년 3월 20일,https://scontent-gmp1-1.cdninstagram.com/v/t51...,🎶,['l4l'],339,https://www.instagram.com/p/CbUJvJkvyGp/
4,atozzang_g,부산 기장,2022년 3월 20일,https://scontent-gmp1-1.cdninstagram.com/v/t51...,#제주도 까지 갈 필요 없는 #부산 갈치조림맛집⠀⠀⠀⠀⠀⠀⠀#송정 에서도 가까운 거...,"['제주도', '부산', '송정', '제주항통갈치', '제주공항', '제주여행', ...",777,https://www.instagram.com/p/CbUv-bwL_Sy/


In [65]:
geo_tag = df['geo_tag']
geo_insta_vocab = {}
for geo in geo_tag:
  if not geo in geo_insta_vocab:
    geo_insta_vocab[geo] = 0
  geo_insta_vocab[geo]+=1
geo_insta_vocab_sorted = sorted(geo_insta_vocab.items(), key = lambda x: -x[1])
len(geo_insta_vocab_sorted)

1280

In [66]:
print(geo_insta_vocab_sorted[:50])

[(nan, 3954), ('제주도 제주', 515), ('Jeju-do', 383), ('제주도', 250), ('환상의 섬 Jeju Island', 178), ('Jeju Island', 64), ('제주도 서귀포', 63), ('아르떼뮤지엄 Arte Museum', 59), ('푸른섬 제주도', 57), ('산방산 유채꼿밭', 45), ('제주도 애월읍', 42), ('서울', 39), ('우도 牛岛 Udo Island, South Korea', 37), ('스누피가든', 36), ('제주 함덕 서우봉 해변', 30), ('제주 협재 해수욕장', 30), ('Seogwipo, Jeju Island, South Korea', 28), ('Jeju, Korea', 28), ('Jeju Island, Korea', 26), ('사계 해변', 24), ('신창풍차해안도로', 24), ('제주 산방산', 21), ('제주도 서귀포시 성산읍', 21), ('Grand Hyatt Jeju', 21), ('미주보석', 21), ('광치기해변', 20), ('성산일출봉', 20), ('섭지코지 (Seopjikoji)', 20), ('부산', 19), ('금악오름', 18), ('비밀의숲', 18), ('금오름', 17), ('금능해수욕장', 16), ('서귀포', 16), ('제주도 어딘가', 16), ('월정리해변', 16), ('용머리해안', 16), ('우도', 16), ('이호테우해변', 15), ('블루보틀 제주', 15), ('해발1950m 한라산 백록담', 14), ('함덕해수욕장', 14), ('제주도 제주시', 14), ('카페 노티드 제주 애월 - Cafe Knotted Jeju Aewol', 14), ('제주 노형수퍼마켙', 14), ('제주', 13), ('유채꽃밭', 13), ('카멜리아 힐', 13), ('Gwangju, South Korea', 12), ('이호테우 목마등대', 12)]


In [67]:
hash_tag = df['hash_tag']
hash_insta_vocab = {}
for hash in hash_tag:
  hash = hash.replace(']','')
  hash = hash.replace('[','')
  hash = hash.replace('\'','')
  li = hash.split(',')
  for h in li:
    h = h.strip()
    if not h in hash_insta_vocab:
      hash_insta_vocab[h]=0
    hash_insta_vocab[h]+=1
hash_insta_vocab_sorted = sorted(hash_insta_vocab.items(), key = lambda x : -x[1])
len(hash_insta_vocab_sorted)

20164

In [68]:
print(hash_insta_vocab_sorted[:100])

[('제주도', 5185), ('', 2771), ('제주', 1185), ('제주도여행', 938), ('여행', 832), ('제주여행', 827), ('jeju', 729), ('일상', 726), ('좋아요', 478), ('여행스타그램', 422), ('맞팔', 413), ('daily', 386), ('소통', 376), ('제주살이', 324), ('jejuisland', 323), ('제주맛집', 300), ('좋반', 299), ('ootd', 273), ('좋아요반사', 272), ('제주도맛집', 271), ('데일리', 265), ('셀카', 262), ('선팔', 251), ('제주카페', 245), ('selfie', 240), ('팔로우', 237), ('서귀포', 233), ('바다', 221), ('셀피', 219), ('유채꽃', 215), ('사진', 214), ('맛집', 191), ('travel', 183), ('먹스타그램', 179), ('부산', 175), ('힐링', 170), ('셀스타그램', 169), ('카페', 168), ('fff', 166), ('일상스타그램', 165), ('애월', 160), ('제주도가볼만한곳', 159), ('선팔하면맞팔', 157), ('f4f', 150), ('데일리룩', 149), ('제주도카페', 148), ('제주핫플', 142), ('제주도민', 141), ('korea', 141), ('해운대', 139), ('오오티디', 138), ('서울', 136), ('봄', 136), ('인친', 131), ('제주일상', 123), ('한라산', 120), ('여행에미치다', 120), ('제주가볼만한곳', 120), ('제주시', 119), ('우도', 117), ('likeforlikes', 109), ('성산일출봉', 109), ('instagood', 107), ('jejudo', 107), ('가족여행', 106), ('맛스타그램', 106), ('풍경', 105),